# 02 — Lasso feature discovery (Omitted Variable Bias)

Identify high-impact omitted variables from 1,200+ raw NHGIS columns using **Lasso (L1) regularization**. All screening logic lives in `scripts/advanced_metrics.run_lasso_feature_selection`; this notebook only runs it and displays results.

- **Raw data**: read-only from `data/raw/nhgis/` (no source CSVs modified).
- **Output**: `output/lasso_feature_shortlist.csv` with NHGIS codes, standardized coefficients, and codebook mapping (B25014 Overcrowding, B25070 Rent squeeze prioritized).

In [42]:
import os
import sys
import pandas as pd

REPO_ROOT = os.path.dirname(os.getcwd()) if os.path.basename(os.getcwd()) == "notebooks" else os.getcwd()
sys.path.insert(0, os.path.join(REPO_ROOT, "scripts"))
DATA_DIR = os.path.join(REPO_ROOT, "data")
OUTPUT_DIR = os.path.join(REPO_ROOT, "output")
os.makedirs(OUTPUT_DIR, exist_ok=True)

from advanced_metrics import run_lasso_feature_selection, NHGIS_CODEBOOK

## Load and examine codebook

Codebook is built from row 2 of each raw NHGIS CSV (variable descriptions). Columns whose description starts with "Margins of error" are excluded from Lasso.

In [ ]:
import os
import glob
import pandas as pd

REPO_ROOT = os.path.dirname(os.getcwd()) if os.path.basename(os.getcwd()) == "notebooks" else os.getcwd()
DATA_DIR = os.path.join(REPO_ROOT, "data")
raw_nhgis = os.path.join(DATA_DIR, "raw", "nhgis")
if not os.path.isdir(raw_nhgis):
    raw_nhgis = DATA_DIR
nhgis_pattern = os.path.join(raw_nhgis, "nhgis*.csv")
nhgis_files = sorted(glob.glob(nhgis_pattern))

# Build codebook from row 0 (codes) and row 1 (descriptions) of each CSV (same logic as scripts/ingest_nhgis.load_nhgis_codebook)
codebook = {}
exclude_set = set()
for filepath in nhgis_files:
    head = pd.read_csv(filepath, header=None, nrows=2, low_memory=False)
    if head.shape[0] < 2:
        continue
    names = head.iloc[0].astype(str).tolist()
    descs = head.iloc[1].tolist()
    n = min(len(names), len(descs))
    for i in range(n):
        col = names[i]
        desc = descs[i] if pd.notna(descs[i]) else ""
        desc_str = str(desc).strip()
        if col not in codebook:
            codebook[col] = desc_str
        if desc_str.lower().startswith("margins of error"):
            exclude_set.add(col)

# As a DataFrame for easier inspection
codebook_df = pd.DataFrame([
    {"nhgis_code": k, "description": v, "is_margin_of_error": k in exclude_set}
    for k, v in codebook.items()
])
codebook_df = codebook_df.sort_values("nhgis_code").reset_index(drop=True)

# Examine
display(codebook_df)

## Run Lasso feature selection

Loads raw NHGIS from `data/raw/nhgis/`, computes **extended_fam_hh_rate** (sum of non-nuclear relatives / householders from B09019, decimal), takes top 100 columns by correlation with the target, runs **LassoCV** (standardized), and writes the top 30 non-zero coefficients to `output/lasso_feature_shortlist.csv`. **Restart kernel and re-run from the top** if you changed the ingestion script so the load cell uses the updated target.

In [44]:
from ingest_nhgis import load_raw_nhgis_wide

_out = load_raw_nhgis_wide(DATA_DIR)
df_wide = _out[0] if isinstance(_out, tuple) else _out
df_wide.shape

/Users/elyas/vscode/capstone_multigen_housing_econometric_analysis/scripts/ingest_nhgis.py:54: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  ns["GEOID"] = ns["STATEA"] + ns["COUNTYA"] + ns["TRACTA"]
/Users/elyas/vscode/capstone_multigen_housing_econometric_analysis/scripts/ingest_nhgis.py:56: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  ns["COUNTY_GEOID"] = ns["STATEA"] + ns["COUNTYA"]
/Users/elyas/vscode/capstone_multigen_housing_econometric_analysis/scripts/ingest_nhgis.py:60: PerformanceWarning: DataFrame is highly fragmente

(85382, 1645)

In [50]:
# Target: extended_fam_hh_rate (relatives per HH). If missing, restart kernel, re-run load cell, and ensure NHGIS includes B09019 (AU46E003, AU46E018–023).
if "extended_fam_hh_rate" in df_wide.columns:
    df_wide["extended_fam_hh_rate"].describe()
else:
    print("extended_fam_hh_rate not in df_wide. Re-run the load cell above (and restart kernel if needed). NHGIS must include table B09019 (AU46E003 + kinship cols).")

extended_fam_hh_rate not in df_wide. Re-run the load cell above (and restart kernel if needed). NHGIS must include table B09019 (AU46E003 + kinship cols).


In [48]:
df_wide['extended_fam_hh_rate'].describe() 

KeyError: 'extended_fam_hh_rate'

In [34]:
df_wide.head()

,GISJOIN,YEAR,STUSAB,REGIONA,DIVISIONA,STATE,STATEA,COUNTY,COUNTYA,COUSUBA,PLACEA,TRACTA,BLKGRPA,CONCITA,AIANHHA,RES_ONLYA,TRUSTA,AIHHTLI,AITSA,ANRCA,CBSAA,CSAA,METDIVA,UAA,CDCURRA,SLDUA,SLDLA,ZCTA5A,SUBMCDA,SDELMA,SDSECA,SDUNIA,PCI,PUMAA,GEO_ID,BTTRA,BTBGA,TL_GEO_ID,NAME_E,AUOVE001,AUOVE002,AUOVE003,AUOVE004,AUOVE005,AUOVE006,AUOVE007,AUOVE008,AUOVE009,AUOVE010,AUOVE011,AUOVE012,AUOVE013,AUOVE014,AUOVE015,AUOVE016,AUOVE017,AUOVE018,AUOVE019,AUOVE020,AUOVE021,AUOVE022,AUOVE023,AUOVE024,AUOVE025,AUOVE026,AUOVE027,AUOVE028,AUOVE029,AUOVE030,AUOVE031,AUOVE032,AUOVE033,AUOVE034,AUOVE035,AUOVE036,AUOVE037,AUOVE038,AUOVE039,AUOVE040,AUOVE041,AUOVE042,AUOVE043,AUOVE044,AUOVE045,AUOVE046,AUOVE047,AUOVE048,AUOVE049,AUOWE001,AUOWE002,AUOWE003,AUO6E001,AUO7E001,AUO7E002,AUO7E003,AUO7E004,AUO7E005,AUO7E006,AUO7E007,AUO7E008,AUO7E009,AUO7E010,AUPCE001,AUPFE001,AUPFE002,AUPFE003,AUPFE004,AUPFE005,AUPFE006,AUPFE007,AUPFE008,AUPFE009,AUPFE010,AUPFE011,AUPFE012,AUPFE013,AUPFE014,AUPFE015,AUPFE016,AUPFE017,AUPFE018,AUPFE019,AUPFE020,AUPFE021,AUPGE001,AUPGE002,AUPGE003,AUPHE001,AUPHE002,AUPHE003,AUPHE004,AUPHE005,AUPHE006,AUPHE007,AUPHE008,AUPHE009,AUPHE010,AUPHE011,AUPHE012,AUPHE013,AUPHE014,AUPWE001,AUPWE002,AUPWE003,AUPWE004,AUPWE005,AUPWE006,AUPWE007,AUPWE008,AUPWE009,AUPWE010,AUPWE011,AUPWE012,AUPWE013,AUPWE014,AUPWE015,AUPWE016,AUPWE017,AUPWE018,AUPWE019,AUPWE020,AUPWE021,AUPYE001,AUPYE002,AUPYE003,AUPYE004,AUPYE005,AUPYE006,AUPYE007,AUPYE008,AUPYE009,AUPYE010,AUPYE011,AUPYE012,AUPYE013,AUPZE001,AUPZE002,AUPZE003,AUPZE004,AUPZE005,AUPZE006,AUPZE007,AUPZE008,AUPZE009,AUPZE010,AUPZE011,AUPZE012,AUPZE013,AUPZE014,AUPZE015,AUPZE016,AUPZE017,AUPZE018,AUPZE019,AUPZE020,AUP1E001,AUP1E002,AUP1E003,AUP1E004,AUP1E005,AUP1E006,AUP1E007,AUP1E008,AUP1E009,AUQNE001,AUQNE002,AUQNE003,AUQNE004,AUQNE005,AUQNE006,AUQNE007,AUQNE008,AUQNE009,AUQNE010,AUQNE011,AUQNE012,AUQNE013,AUQNE014,AUQNE015,AUQNE016,AUQNE017,AUQNE018,AUQNE019,AUQRE001,AUQRE002,AUQRE003,AUQRE004,AUQRE005,AUQRE006,AUQRE007,AUQRE008,AUQRE009,AUQRE010,AUQRE011,AUQRE012,AUQRE013,AUQRE014,AUQRE015,AUQRE016,AUQRE017,AUQTE001,AUQTE002,AUQTE003,AUQTE004,AUQTE005,AUQTE006,AUQTE007,AUQTE008,AUQTE009,AUQTE010,AUQTE011,AUQTE012,AUQTE013,AUQTE014,AUQTE015,AUQTE016,AUQ8E001,AUQ8E002,AUQ8E003,AUQ8E004,AUQ8E005,AUQ8E006,AUQ8E007,AUQ8E008,AUQ8E009,AUQ8E010,AUQ8E011,AUQ8E012,AUQ8E013,AUQ8E014,AUQ8E015,AUQ8E016,AUQ8E017,AUQ8E018,AUQ8E019,AUQ8E020,AUQ8E021,AUQ8E022,AUQ8E023,AUQ8E024,AUQ8E025,AURLE001,AURLE002,AURLE003,AURLE004,AURLE005,AURLE006,AURLE007,AURLE008,AURLE009,AURLE010,AURLE011,AURLE012,AURLE013,AURLE014,AURNE001,AURNE002,AURNE003,AURNE004,AURNE005,AURNE006,AURNE007,AURNE008,AURRE001,AURRE002,AURRE003,AURRE004,AURRE005,AURRE006,AURRE007,AURRE008,AURRE009,AURRE010,AURRE011,AURRE012,AURRE013,AURRE014,AURRE015,AURRE016,AURRE017,AURRE018,AURRE019,AURRE020,AURRE021,AURRE022,AURRE023,AURRE024,AURRE025,AURRE026,AURRE027,AURRE028,AURRE029,AURRE030,AURRE031,AURRE032,AURRE033,AURRE034,AURRE035,AURTE001,AURTE002,AURTE003,AURTE004,AURTE005,AURTE006,AURTE007,AURTE008,AURTE009,AURTE010,AURTE011,AURTE012,AURTE013,AURTE014,AURTE015,AURTE016,AURTE017,AURUE001,AURVE001,AUSEE001,AUSEE002,AUSEE003,AUSYE001,AUS0E001,AUS6E001,AUS7E001,AUTPE001,AUTPE002,AUTPE003,AUTPE004,AUTPE005,AUTPE006,AUTPE007,AUTWE001,AUTWE002,AUTWE003,AUTWE004,AUTWE005,AUTWE006,AUTWE007,AUUCE001,AUUDE001,AUUDE002,AUUDE003,AUUEE001,AUUEE002,AUUEE003,AUUGE001,AUUGE002,AUUGE003,AUUNE001,AUUNE002,AUUNE003,AUUOE001,AUUOE002,AUUOE003,AUUOE004,AUUOE005,AUUOE006,AUUOE007,AUUOE008,AUUPE001,AUUPE002,AUUPE003,AUUPE004,AUUPE005,AUUPE006,AUUPE007,AUUPE008,AUUPE009,AUUPE010,AUUTE001,AUUTE002,AUUTE003,AUUVE001,AUUVE002,AUUVE003,AUUZE001,AUUZE002,AUUZE003,AUU0E001,AUU0E002,AUU0E003,AUU1E001,AUU1E002,AUU1E003,AUU1E004,AUU1E005,AUU1E006,AUU1E007,AUU1E008,AUU1E009,AUU1E010,AUU1E011,AUU1E012,AUU1E013,AUU1E014,AUU1E015,AUU1E016,AUU1E017,AUVFE001,AUVFE002,AUVFE003,AUVFE004,AUVFE005,AUVFE006,AUVFE007,AUVFE008,AUVFE009,AUVFE010,AUVGE001,AUVLE001,AUVLE002,AUVLE003,AUVLE004

In [40]:
#print summary statistics for multigen_rate
df_wide["AU46E002"].describe()

count    85382.000000
mean      3863.146401
std       1828.021962
min          0.000000
25%       2624.000000
50%       3674.000000
75%       4874.000000
max      44356.000000
Name: AU46E002, dtype: float64

In [ ]:
df_wide["AU46E002"].describe()

In [33]:
# Calculate percent missing for each column, sort descending, and display as scrollable DataFrame
import pandas as pd

pd.set_option('display.max_rows', 2000)  # Increase the max number of rows to scroll
missing_pct = (df_wide.isnull().sum() / len(df_wide) * 100)
missing_pct_sorted = missing_pct.sort_values(ascending=False)
display(missing_pct_sorted)

AUYOM004         100.000000
SDUNIA           100.000000
AU50M002         100.000000
CDCURRA          100.000000
SLDUA            100.000000
SLDLA            100.000000
ZCTA5A           100.000000
SUBMCDA          100.000000
SDELMA           100.000000
SDSECA           100.000000
PCI              100.000000
CSAA             100.000000
PUMAA            100.000000
AU50E003         100.000000
BTTRA            100.000000
BTBGA            100.000000
AU50M003         100.000000
AU50E002         100.000000
AU50E001         100.000000
AUYOM005         100.000000
METDIVA          100.000000
UAA              100.000000
CBSAA            100.000000
PLACEA           100.000000
AUYOM003         100.000000
REGIONA          100.000000
DIVISIONA        100.000000
AUYOM002         100.000000
AUYOM001         100.000000
AUYOM006         100.000000
ANRCA            100.000000
COUSUBA          100.000000
AUYOM007         100.000000
BLKGRPA          100.000000
CONCITA          100.000000
AIANHHA          100

In [16]:
results = run_lasso_feature_selection(
    data_dir=DATA_DIR,
    output_path=os.path.join(OUTPUT_DIR, "lasso_feature_shortlist.csv"),
    target_col="extended_fam_hh_rate",
    top_corr_n=100,
    top_nonzero_n=30,
    n_alphas=100,
    cv=5,
)

print(f"Optimal Lasso alpha: {results['optimal_alpha']:.6f}")
print(f"Shortlist written to: {results['output_path']}")
print(f"Top 30 non-zero features: {len(results['shortlist_codes'])}")

/Users/elyas/vscode/capstone_multigen_housing_econometric_analysis/scripts/ingest_nhgis.py:54: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  ns["GEOID"] = ns["STATEA"] + ns["COUNTYA"] + ns["TRACTA"]
/Users/elyas/vscode/capstone_multigen_housing_econometric_analysis/scripts/ingest_nhgis.py:56: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  ns["COUNTY_GEOID"] = ns["STATEA"] + ns["COUNTYA"]
/Users/elyas/vscode/capstone_multigen_housing_econometric_analysis/scripts/ingest_nhgis.py:60: PerformanceWarning: DataFrame is highly fragmente

Optimal Lasso alpha: 0.007599
Shortlist written to: /Users/elyas/vscode/capstone_multigen_housing_econometric_analysis/output/lasso_feature_shortlist.csv
Top 30 non-zero features: 30


## Shortlist: NHGIS codes and standardized coefficients

Exact NHGIS codes with standardized coefficient values. **Codebook mapping** prioritizes Table **B25014** (Occupants per room / Overcrowding) and **B25070** (Gross rent as % of income / Economic squeeze).

In [17]:
shortlist = results["shortlist"]
display(shortlist)

,nhgis_code,standardized_coef,abs_coef,codebook_table
0,AU46E026,-7.612892,7.612892,B11017 — Household type (incl. multigenerational)
1,AVA1E001,1.972548,1.972548,B19083 — Gini index
2,AUOVE007,1.560446,1.560446,B01001 — Sex by age
3,AU46E002,1.171221,1.171221,B11017 — Household type (incl. multigenerational)
4,AUOVM007,-0.981149,0.981149,— (see NHGIS codebook)
5,AUVGE001,0.866782,0.866782,— (see NHGIS codebook)
6,AURNE008,-0.770895,0.770895,— (see NHGIS codebook)
7,AUPWM019,-0.721026,0.721026,— (see NHGIS codebook)
8,AUPWE019,0.683256,0.683256,B08301 — Commute
9,AUOVE031,0.657380,0.657380,B01001 — Sex by age


In [18]:
# Highlight rows that map to B25014 (Overcrowding) or B25070 (Rent squeeze)
b25014 = shortlist["codebook_table"].str.contains("B25014", na=False)
b25070 = shortlist["codebook_table"].str.contains("B25070", na=False)
print("Rows mapping to B25014 (Overcrowding):")
print(shortlist.loc[b25014].to_string())
print()
print("Rows mapping to B25070 (Rent squeeze):")
print(shortlist.loc[b25070].to_string())

Rows mapping to B25014 (Overcrowding):
Empty DataFrame
Columns: [nhgis_code, standardized_coef, abs_coef, codebook_table]
Index: []

Rows mapping to B25070 (Rent squeeze):
Empty DataFrame
Columns: [nhgis_code, standardized_coef, abs_coef, codebook_table]
Index: []


## Top 3 for schema integration

Copy the **nhgis_code** values below into `scripts/core_metrics.py`: add them to `ANALYSIS_READY_SCHEMA["feature_cols"]` and add human-readable labels to `FEATURE_LABELS`. Raw NHGIS columns are preserved in the wide merge, so these codes will be available in analysis-ready data when included in the schema.

In [19]:
top3 = shortlist.head(3)
print("Top 3 Lasso-selected variables (add to core_metrics.py):")
for _, row in top3.iterrows():
    print(f"  {row['nhgis_code']}: {row['codebook_table']}  (std coef = {row['standardized_coef']:.4f})")

Top 3 Lasso-selected variables (add to core_metrics.py):
  AU46E026: B11017 — Household type (incl. multigenerational)  (std coef = -7.6129)
  AVA1E001: B19083 — Gini index  (std coef = 1.9725)
  AUOVE007: B01001 — Sex by age  (std coef = 1.5604)
